In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt

import sys
sys.path.append('../src/')
import dictionaries
import omnifoldHI

In [ ]:
dict_data        = dictionaries.load('../analysis_4v5/dict/dict_data_h110_p000.pkl')
multifoldHI_out  = dictionaries.load('../analysis_4v5/dict/multifoldHI_out_h110_p000_mfHI17_it8_bce_hl2x100_bs8192.pkl')

In [ ]:
cmap = plt.get_cmap('plasma')

with mpl.rc_context({'font.family': 'sans-serif', 'font.size': 14, 'text.usetex':False}):
    _, ((ax1, ax2),
        (ax3, ax4)) = plt.subplots(2, 2, figsize=(14,10))

    n_it = len(multifoldHI_out['train_loss'])
    for it in range(1, n_it):

        ax1.plot(multifoldHI_out['train_loss'][it,0], c=cmap(it/n_it), label=f'it {it}')
        ax2.plot(multifoldHI_out['train_loss'][it,1], c=cmap(it/n_it))

        ax1.plot(multifoldHI_out['val_loss'][it,0], ls='--', c=cmap(it/n_it))
        ax2.plot(multifoldHI_out['val_loss'][it,1], ls='--', c=cmap(it/n_it))

        ax3.plot(multifoldHI_out['train_accuracy'][it,0], c=cmap(it/n_it))
        ax4.plot(multifoldHI_out['train_accuracy'][it,1], c=cmap(it/n_it))

        ax3.plot(multifoldHI_out['val_accuracy'][it,0], ls='--', c=cmap(it/n_it))
        ax4.plot(multifoldHI_out['val_accuracy'][it,1], ls='--', c=cmap(it/n_it))

    ax1.legend()
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('Loss')
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('Loss')
    ax3.set_xlabel('epoch')
    ax3.set_ylabel('Accuracy')
    ax4.set_xlabel('epoch')
    ax4.set_ylabel('Accuracy')

    ax1.set_title('Step I')
    ax2.set_title('Step II')

    plt.show()

In [ ]:
dict_multifoldHI_list = []
for it in range(len(multifoldHI_out['train_loss'])):
    print('it:',it)
    dict_multifoldHI = omnifoldHI.create_dict_multifoldHI(dict_data,
                                                          multifoldHI_out['weights_stat'],
                                                          multifoldHI_out['weights_syst'],
                                                          multifold_it=it)
    dict_multifoldHI_list.append(dict_multifoldHI)

multifoldHI_obs = [
    "pt_jet", "eta_jet", "phi_jet", "m_jet", "n_jet",
    "z_DyG", "th_DyG", "kt_DyG",
    "pt_SD", "m_SD", "pt_RSD", "n_RSD",
    "tau1", "tau2", "tau3", "tau4", "tau5"
]
plot_obs = dict_data.keys()

nature_label = ''
synthetic_label = ''

# Plot
print('\nPlotting.')
cmap = plt.get_cmap('plasma')
with mpl.rc_context({'font.family': 'sans-serif', 'font.size': 14, 'text.usetex':False}):

    hist_style = {
        'gen': {'label':'gen', 'lw':2, 'color':'royalblue'},
        'sim': {'label':'sim', 'lw':2, 'color':'orange'},
        'tru': {'label':'Truth', 'lw':2, 'color':'green'},
        'dat': {'label':'dat', 'lw':2, 'color':'k'},
        'omnifold': {'label':'Test OmniFold', 'lw':1.5, 'capsize':2},
        'omnifold_syst': {'alpha':.3},
    }

    # Observables to plot
    obs_plot_multifoldHI = multifoldHI_obs
    obs_plot_all         = plot_obs

    # Create figure and grid
    fig = plt.figure(figsize=(25,20/4*int(np.ceil(len(obs_plot_all)/4))))
    gs_main = fig.add_gridspec(int(np.ceil(len(obs_plot_all)/4)), 4, wspace=.45)

    # Plot for each observable
    axes = []
    for i, key in enumerate(obs_plot_all):

        obs_dict_data = dict_data[key]

        #fig = plt.figure(figsize=(7,6))

        gs = gs_main[i].subgridspec(2,2, height_ratios=(3,1), width_ratios=(5,1), hspace=0, wspace=.03)

        axy   = fig.add_subplot(gs[0,0])
        axr   = fig.add_subplot(gs[1,0], sharex=axy)
        axFTy = fig.add_subplot(gs[0,1])
        axFTr = fig.add_subplot(gs[1,1], sharex=axFTy, sharey=axr)
        axes.append({'axy':axy, 'axr':axr, 'axFTy':axFTy, 'axFTr':axFTr})

        # BASELINE
        ## Plot datasets
        axy.stairs(obs_dict_data['hist_nat_E'], obs_dict_data['bins_E'], baseline=0, fill=True, alpha=.2, color='k', label='Measured')
        axy.stairs(obs_dict_data['hist_nat_C'], obs_dict_data['bins_C'], baseline=0, fill=True, alpha=.4, color='green', label='Truth')

        ###
        # Plot fake
        axFTy.stairs(obs_dict_data['hist_nat_F'], [.6,1.4], baseline=0, fill=True, alpha=.4, color='green')

        ###
        # Plot trash
        axFTy.stairs(obs_dict_data['hist_nat_T'], [1.6,2.4], baseline=0, fill=True, alpha=.2, color='k')

        ###
        # Compute raito to truth
        tru_tru     = (obs_dict_data['hist_nat_C']+1e-10)/(obs_dict_data['hist_nat_C']+1e-10)
        tru_tru_std = (obs_dict_data['hist_nat_C_unc'])/(obs_dict_data['hist_nat_C']+1e-10)

        ###
        # Plot ratio to truth
        axr.errorbar(obs_dict_data['midbin_C'], np.ones(len(obs_dict_data['bins_C'])-1), 0,obs_dict_data['midbin_C_unc'], **hist_style['tru'])
        axr.stairs(tru_tru+tru_tru_std, obs_dict_data['bins_C'], baseline=1, fill=True, alpha=.4, color='green')
        axr.stairs(tru_tru-tru_tru_std, obs_dict_data['bins_C'], baseline=1, fill=True, alpha=.4, color='green')

        ###
        # Compute fake ratio to truth
        fake_tru_tru     = (obs_dict_data['hist_nat_F']+1e-10)/(obs_dict_data['hist_nat_F']+1e-10)
        fake_tru_tru_std = (obs_dict_data['hist_nat_F_unc']+1e-10)/(obs_dict_data['hist_nat_F']+1e-10)

        ###
        # Plot fake ratio to truth
        axFTr.errorbar([1], [1], 0, [.4], **hist_style['tru'])
        axFTr.stairs(fake_tru_tru-fake_tru_tru_std, [.6,1.4], baseline=1, fill=True, alpha=.4, color='green')
        axFTr.stairs(fake_tru_tru+fake_tru_tru_std, [.6,1.4], baseline=1, fill=True, alpha=.4, color='green')

        ###
        # Compute trash ratio to truth
        tras_dat_dat     = (obs_dict_data['hist_nat_T']+1e-10)/(obs_dict_data['hist_nat_T']+1e-10)
        tras_dat_dat_std = (obs_dict_data['hist_nat_T_unc']+1e-10)/(obs_dict_data['hist_nat_T']+1e-10)

        ###
        # Plot trash ratio to truth
        axFTr.errorbar([2], [1], 0, [.4], **hist_style['dat'])
        axFTr.stairs(fake_tru_tru-fake_tru_tru_std, [1.6,2.4], baseline=1, fill=True, alpha=.2, color='k')
        axFTr.stairs(fake_tru_tru+fake_tru_tru_std, [1.6,2.4], baseline=1, fill=True, alpha=.2, color='k')

        if key in obs_plot_multifoldHI:

            n_it = len(dict_multifoldHI_list)
            for it, dict_multifoldHI in enumerate(dict_multifoldHI_list):

                obs_dict_multifoldHI = dict_multifoldHI[key]

                # omnifold (original)
                # Plot unfolded
                axy.errorbar(obs_dict_multifoldHI['midbin_C'], obs_dict_multifoldHI['hist_unf_C'], obs_dict_multifoldHI['hist_unf_C_unc_stat'], obs_dict_multifoldHI['midbin_C_unc'], fmt=',', color=cmap(it/n_it), **hist_style['omnifold'])
                axy.bar(obs_dict_multifoldHI['midbin_C'], 2*obs_dict_multifoldHI['hist_unf_C_unc_stat'], 2*obs_dict_multifoldHI['midbin_C_unc'], obs_dict_multifoldHI['hist_unf_C']-obs_dict_multifoldHI['hist_unf_C_unc_stat'], color=cmap(it/n_it), **hist_style['omnifold_syst'])
                
                ###
                # Compute unfolded raito to truth
                omn_tru          = (obs_dict_multifoldHI['hist_unf_C']+1e-10)/(obs_dict_data['hist_nat_C']+1e-10)
                omn_tru_unc_stat = (obs_dict_multifoldHI['hist_unf_C_unc_stat'])/(obs_dict_data['hist_nat_C']+1e-10)
                omn_tru_unc_syst = (obs_dict_multifoldHI['hist_unf_C_unc_syst'])/(obs_dict_data['hist_nat_C']+1e-10)

                ###
                # Plot unfolded ratio to truth
                axr.errorbar(obs_dict_multifoldHI['midbin_C'], omn_tru, omn_tru_unc_stat, obs_dict_multifoldHI['midbin_C_unc'], color=cmap(it/n_it), **hist_style['omnifold'])
                axr.bar(obs_dict_multifoldHI['midbin_C'], 2*omn_tru_unc_syst, 2*obs_dict_multifoldHI['midbin_C_unc'], omn_tru-omn_tru_unc_syst, color=cmap(it/n_it), **hist_style['omnifold_syst'])

                ###
                # Plot unfolded fake
                axFTy.errorbar([1], obs_dict_multifoldHI['hist_unf_F'], obs_dict_multifoldHI['hist_unf_F_unc_stat'], [.4], color=cmap(it/n_it), **hist_style['omnifold'])
                axFTy.bar([1], 2*obs_dict_multifoldHI['hist_unf_F_unc_stat'], [2*.4], obs_dict_multifoldHI['hist_unf_F']-obs_dict_multifoldHI['hist_unf_F_unc_stat'], color=cmap(it/n_it), **hist_style['omnifold_syst'])

                ###
                # Compute unfolded fake ratio to truth
                fake_omn_tru          = (obs_dict_multifoldHI['hist_unf_F']+1e-10)/(obs_dict_data['hist_nat_F']+1e-10)
                fake_omn_tru_unc_stat = (obs_dict_multifoldHI['hist_unf_F_unc_stat']+1e-10)/(obs_dict_data['hist_nat_F']+1e-10)
                fake_omn_tru_unc_syst = (obs_dict_multifoldHI['hist_unf_F_unc_syst']+1e-10)/(obs_dict_data['hist_nat_F']+1e-10)

                ###
                # Plot unfolded fake ratio to truth
                axFTr.errorbar([1], fake_omn_tru, fake_omn_tru_unc_stat, [.4], color=cmap(it/n_it), **hist_style['omnifold'])
                axFTr.bar([1], 2*fake_omn_tru_unc_syst, [2*.4], fake_omn_tru-fake_omn_tru_unc_syst, color=cmap(it/n_it), **hist_style['omnifold_syst'])

                ###
                # Plot unfolded trash
                axFTy.errorbar([2], obs_dict_multifoldHI['hist_unf_T'], obs_dict_multifoldHI['hist_unf_T_unc_stat'], [.4], color=cmap(it/n_it), **hist_style['omnifold'])
                axFTy.bar([2], 2*obs_dict_multifoldHI['hist_unf_T_unc_stat'], [2*.4], obs_dict_multifoldHI['hist_unf_T']-obs_dict_multifoldHI['hist_unf_T_unc_stat'], color=cmap(it/n_it), **hist_style['omnifold_syst'])

                ###
                # Compute unfolded trash ratio to truth
                tras_omn_dat          = (obs_dict_multifoldHI['hist_unf_T']+1e-10)/(obs_dict_data['hist_nat_T']+1e-10)
                tras_omn_dat_unc_stat = (obs_dict_multifoldHI['hist_unf_T_unc_stat']+1e-10)/(obs_dict_data['hist_nat_T']+1e-10)
                tras_omn_dat_unc_syst = (obs_dict_multifoldHI['hist_unf_T_unc_syst']+1e-10)/(obs_dict_data['hist_nat_T']+1e-10)

                ###
                # Plot unfolded trash ratio to truth
                axFTr.errorbar([2], tras_omn_dat, tras_omn_dat_unc_stat, [.4], color=cmap(it/n_it), **hist_style['omnifold'])
                axFTr.bar([2], 2*tras_omn_dat_unc_syst, [2*.4], tras_omn_dat-tras_omn_dat_unc_syst, color=cmap(it/n_it), **hist_style['omnifold_syst'])

        
        # Labels and limits
        axr.set_xlabel(obs_dict_data['xlabel'])
        axy.set_ylabel(obs_dict_data['ylabel'])
        axr.set_ylabel('ratio to\ntruth')
        axFTr.set_ylim(.65,1.35)
        axFTr.set_xlim(.3,2.7)

        # Ticks
        axFTr.set_xticks(ticks=[1,2], labels=[r'$F$',r'$T$'])
        axFTr.set_yticks(ticks=[.8,1,1.2])
        for axis in [axy,axr,axFTy,axFTr]:
            axis.minorticks_on()
            axis.tick_params(axis='both', which='both', direction='in', top=True, right=True)
        for axis in [axFTy, axFTr]:
            axis.tick_params(axis='x', which='minor', top=False, bottom=False)
        for axis in [axy, axFTy]:
            axis.tick_params(axis='x', labelbottom=False)
        for axis in [axFTy]:
            axis.tick_params(axis='y', labelleft=False)
            axis.tick_params(axis='y', labelright=True)
        for axis in [axFTr]:
            axis.tick_params(axis='y', labelleft=False)
            axis.tick_params(axis='y', labelright=False)

    axes[-1]['axFTy'].text(2.5,1, f'Nature: {nature_label}', color='gray', fontsize='small', ha='right', va='top', transform=axes[-1]['axFTy'].transAxes, rotation=-90)
    axes[-1]['axFTy'].text(2.1,1, f'Sysnthetic: {synthetic_label}', color='gray', fontsize='small', ha='right', va='top', transform=axes[-1]['axFTy'].transAxes, rotation=-90)

    axes[1]['axy'].text(.04,.96, r'hardest jet in pp@5.20TeV', fontsize='small', ha='left', va='top', transform=axes[1]['axy'].transAxes)
    axes[1]['axy'].text(.04,.89, r'$R_{ak_t}=0.4, |\eta|<2.8$', fontsize='small', ha='left', va='top', transform=axes[1]['axy'].transAxes)

    #axes[1]['axy'].set_ylim(0,12)

    axes[0]['axy'].legend(frameon=False, fontsize='small')

    #plt.savefig(f'figures/analysis_{code}.pdf', bbox_inches='tight')
    plt.show()

    print('\nDone.')